# 08 - Python Finance

**Capitulo 08**: Análise de empresas com Python, obtendo dados das empresas listadas na bolsa usando Python.

Realizar webscraping (raspagem de dados) e baixar os dados dos balanços das empresas disponíveis no site **fundamentus**.

Em seguida criar um ranking das melhores empresas segundo a da Magic Formula de **Joel Greenblatt**.

## **Configurações Iniciais**

## 1. Importando bibliotecas

In [1]:
import numpy as np
import pandas as pd
import string
import warnings
warnings.filterwarnings('ignore')

import requests

## 2. Obtendo e tratando os dados

In [2]:
url = 'http://www.fundamentus.com.br/resultado.php'

In [3]:
header = {
  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"  
 }

r = requests.get(url, headers=header)

In [4]:
df = pd.read_html(r.text,  decimal=',', thousands='.')[0]

Exibindo dados

In [5]:
df

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,POPR4,10.17,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"8,66%","5,65%",1.08,"15,25%","19,93%",0.0,5.458030e+08,0.82,"30,93%"
1,VNET3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","5,17%",0.0,9.490850e+09,0.00,"-17,77%"
2,PMET3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","4,10%",0.0,-2.908630e+08,0.00,"37,74%"
3,MRSA6B,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","9,98%",0.0,4.312820e+09,0.00,"1,03%"
4,CLAN3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","-1,05%",0.0,1.012240e+09,0.00,"-63,96%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,LWSA3,23.00,682.15,19.98,27.582,"0,12%",9.272,37.73,311.15,462.67,304.88,132.11,"8,86%","4,04%",1.79,"4,30%","2,93%",270431000.0,6.739730e+08,0.23,"24,50%"
937,PRBC4,14.54,808.37,39.12,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","4,84%",0.0,1.210880e+09,0.00,"11,60%"
938,UBBR11,14.75,1201.81,3.91,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.0,1.031720e+10,0.00,"10,58%"
939,UBBR3,18.00,1466.61,4.77,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.0,1.031720e+10,0.00,"10,58%"


In [6]:
for coluna in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
  df[coluna] = df[coluna].str.replace('.', '')
  df[coluna] = df[coluna].str.replace(',', '.')
  df[coluna] = df[coluna].str.rstrip('%').astype('float') / 100

Exibindo dados tratados

In [7]:
df

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,POPR4,10.17,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0866,0.0565,1.08,0.1525,0.1993,0.0,5.458030e+08,0.82,0.3093
1,VNET3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0517,0.0,9.490850e+09,0.00,-0.1777
2,PMET3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0410,0.0,-2.908630e+08,0.00,0.3774
3,MRSA6B,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0998,0.0,4.312820e+09,0.00,0.0103
4,CLAN3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,-0.0105,0.0,1.012240e+09,0.00,-0.6396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,LWSA3,23.00,682.15,19.98,27.582,0.0012,9.272,37.73,311.15,462.67,304.88,132.11,0.0886,0.0404,1.79,0.0430,0.0293,270431000.0,6.739730e+08,0.23,0.2450
937,PRBC4,14.54,808.37,39.12,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0484,0.0,1.210880e+09,0.00,0.1160
938,UBBR11,14.75,1201.81,3.91,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0033,0.0,1.031720e+10,0.00,0.1058
939,UBBR3,18.00,1466.61,4.77,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0033,0.0,1.031720e+10,0.00,0.1058


## 2. Analisando os dados coletados


In [8]:
df.shape

(941, 21)

Liquidez diaria dos últimos dois meses maior 1k, empresas com negociação média superior a 1k por dia nos últimos dois meses.

In [9]:
df = df[df['Liq.2meses'] > 1000000]

Excluindo empresas pequenas com baixa negociação da análise.

In [10]:
df.shape

(247, 21)

Listar empresas segundo os critérios, **'pos', 'EV/EBIT', 'ROIC'**

In [11]:
ranking = pd.DataFrame()
ranking['pos'] = range(1,151)
ranking['EV/EBIT'] = df[ df['EV/EBIT'] > 0 ].sort_values(by=['EV/EBIT'])['Papel'][:150].values
ranking['ROIC'] = df.sort_values(by=['ROIC'], ascending=False)['Papel'][:150].values #maior para menor

Exibindo dados ranqueados

In [12]:
ranking

,pos,EV/EBIT,ROIC
0,1,PSSA3,WIZS3
1,2,PCAR3,PSSA3
2,3,WIZS3,ODPV3
3,4,MRFG3,MOSI3
4,5,FHER3,CMIN3
...,...,...,...
145,146,JSLG3,CCPR3
146,147,RENT3,RLOG3
147,148,CSAN3,DMVF3
148,149,ELET3,PRIO3


Formula magica somar os dois rankings, quais empresas tem os melhores valores (empresas melhores classificadas com relação ao preço e relação a qualidade).

In [13]:
a = ranking.pivot_table(columns='EV/EBIT', values='pos')

In [14]:
b = ranking.pivot_table(columns='ROIC', values='pos')

In [15]:
t = pd.concat([a,b])
t

,ABEV3,AGRO3,ALSO3,ALUP11,AMBP3,ANIM3,ASAI3,AURA33,B3SA3,BEEF3,BRDT3,BRFS3,BRKM5,BRML3,BRPR3,BSEV3,CAML3,CARD3,CCPR3,CCRO3,CESP6,CIEL3,CMIG3,CMIG4,CMIN3,COCE5,CPFE3,CPLE3,CPLE6,CRFB3,CSAN3,CSMG3,CSNA3,CURY3,CYRE3,DIRR3,DMVF3,DTEX3,ECOR3,EGIE3,...,TECN3,TEND3,TESA3,TGMA3,TIET11,TIET4,TIMS3,TOTS3,TRIS3,TRPL4,TUPY3,UGPA3,UNIP6,USIM3,USIM5,VALE3,VAMO3,VIVA3,VIVT3,VLID3,VVAR3,WIZS3,WSON33,AERI3,ALPA4,BOAS3,GNDI3,HAPV3,LINX3,LREN3,MGLU3,MOSI3,PETZ3,PNVL3,PRIO3,RADL3,RDOR3,SULA11,TFCO4,WEGE3
pos,103,82,104.0,6,122,100,54,51,111,16,116,73,45,128.0,137.0,60,53,71,77,119,105.0,132.0,34,22,25,108.0,59,17,20,64,148,27,50,70,123,49,24,110,48,52,...,83.0,37,44,81,18,19,57,94,68,10,84,134,43,58,56,7,145,142,91,131.0,61,3,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pos,38,94,NaN,36,58,130,17,11,24,20,126,91,61,NaN,NaN,33,54,37,146,121,NaN,NaN,93,92,5,NaN,52,86,84,48,140,62,23,21,143,139,148,100,30,29,...,NaN,59,102,50,15,16,101,14,79,31,127,137,18,82,83,13,60,55,135,NaN,64,1,34,72.0,41.0,74.0,67.0,70.0,128.0,107.0,142.0,4.0,75.0,132.0,149.0,96.0,134.0,106.0,28.0,19.0


In [16]:
# descartar as empresas que não tem os dois rank, indicadores EV/EBIT e ROIC
rank = t.dropna(axis=1).sum()
rank

ABEV3     141
AGRO3     176
ALUP11     42
AMBP3     180
ANIM3     230
         ... 
VIVA3     197
VIVT3     226
VVAR3     125
WIZS3       4
WSON33     60
Length: 133, dtype: int64

## 3. Resultados


In [17]:
df.describe()

,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
count,247.000000,247.000000,247.000000,247.000000,247.000000,247.000000,247.000000,247.000000,247.000000,247.000000,247.000000,247.000000,247.000000,247.000000,247.000000,247.000000,2.470000e+02,2.470000e+02,247.000000,247.000000
mean,23.142996,-38.711700,7.370607,6.728186,0.017175,1.530328,6.991538,10.298381,-0.877004,17.893563,1.384211,-0.119398,-2.198728,1.952874,0.079392,0.034019,1.312121e+08,1.191329e+10,1.814575,0.304950
std,19.860729,658.004994,34.511303,59.997425,0.025437,3.796225,25.386088,139.636454,47.358201,239.930827,111.515298,2.956637,28.882694,1.598856,0.132161,1.895335,3.058425e+08,3.634592e+10,10.267169,2.546868
min,0.870000,-5448.940000,-17.550000,0.000000,0.000000,0.000000,-115.470000,-1261.630000,-248.230000,-1258.160000,-1258.160000,-40.192800,-448.133000,0.000000,-1.131300,-26.264900,1.032290e+06,-1.440710e+10,-19.030000,-0.515500
25%,9.635000,0.000000,1.070000,0.641000,0.000000,0.380500,1.255000,2.525000,-4.350000,3.800000,4.145000,0.019800,0.000000,1.120000,0.009400,0.009900,1.135340e+07,7.802195e+08,0.175000,0.000000
50%,18.820000,10.920000,2.050000,1.430000,0.004500,0.774000,3.880000,8.850000,-1.340000,10.240000,7.520000,0.106500,0.047900,1.680000,0.075900,0.097800,3.748310e+07,2.234260e+09,0.570000,0.070000
75%,28.600000,25.680000,4.255000,3.365000,0.025250,1.331000,9.540000,19.235000,0.000000,20.685000,14.650000,0.209100,0.138750,2.355000,0.121100,0.168650,1.346920e+08,7.532410e+09,1.300000,0.175850
max,153.000000,3690.320000,503.020000,942.787000,0.190600,43.747000,163.770000,1089.840000,462.670000,3194.580000,252.130000,1.023800,1.249000,11.800000,0.622400,6.236300,2.931480e+09,3.084100e+11,151.270000,39.144300


In [18]:
df["Liq.2meses"].describe()

count    2.470000e+02
mean     1.312121e+08
std      3.058425e+08
min      1.032290e+06
25%      1.135340e+07
50%      3.748310e+07
75%      1.346920e+08
max      2.931480e+09
Name: Liq.2meses, dtype: float64

In [19]:
df["EV/EBIT"].describe()

count     247.000000
mean       17.893563
std       239.930827
min     -1258.160000
25%         3.800000
50%        10.240000
75%        20.685000
max      3194.580000
Name: EV/EBIT, dtype: float64

In [20]:
df["ROIC"].describe()

count    247.000000
mean       0.079392
std        0.132161
min       -1.131300
25%        0.009400
50%        0.075900
75%        0.121100
max        0.622400
Name: ROIC, dtype: float64

In [21]:
rank.sort_values()[:38]

PSSA3      3
WIZS3      4
MRFG3     13
PLPL3     15
VALE3     20
CMIN3     30
TIET11    33
TIET4     35
BEEF3     36
TASA4     38
TASA3     40
TRPL4     41
ALUP11    42
PCAR3     53
JBSS3     56
SAPR4     57
SAPR3     57
SAPR11    57
LEVE3     58
WSON33    60
UNIP6     61
AURA33    62
TAEE11    65
TAEE3     65
TAEE4     66
LAVV3     69
ASAI3     71
CSNA3     73
EQTL3     74
ODPV3     75
ECOR3     78
EGIE3     81
CSMG3     89
QUAL3     90
CURY3     91
BSEV3     93
FHER3     94
TEND3     96
dtype: int64

In [22]:
rank.shape

(133,)

Remover empresas seguradoras, empresas de bancos.

Distorções quando comparadas empresas do setor financeiro utilizando os multiplos (indicadores EV/EBIT e ROIC).

## 3.1 Resultados

In [23]:
df = df[df['Liq.2meses'] > 1000000000]
df

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
675,BBDC4,26.80,14.37,1.65,0.000,0.0248,0.000,0.0,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.1151,1.261630e+09,1.437030e+11,0.00,-0.0927
679,ITUB4,28.07,14.51,2.01,0.000,0.0194,0.000,0.0,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.1387,1.233940e+09,1.366990e+11,0.00,-0.2996
749,VALE3,95.53,18.90,2.72,2.421,0.0698,1.056,9.9,4.71,-2.96,4.79,4.14,0.5136,0.1194,1.67,0.2742,0.1438,2.931480e+09,1.857850e+11,0.42,0.2086
861,PETR4,23.46,43.05,0.99,1.125,0.0000,0.310,50.7,3.28,-0.57,6.79,4.05,0.3434,0.0230,1.04,0.1053,0.0230,2.627880e+09,3.084100e+11,1.27,-0.0013


In [24]:
df.shape

(4, 21)

In [25]:
df.describe()

,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
count,4.000000,4.00000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000e+00,4.000000e+00,4.000000,4.000000
mean,43.465000,22.70750,1.842500,0.886500,0.028500,0.341500,15.150000,1.997500,-0.882500,2.895000,2.047500,0.214250,0.035600,0.677500,0.094875,0.105150,2.013732e+09,1.936492e+11,0.422500,-0.046250
std,34.764409,13.72379,0.721543,1.152293,0.029521,0.498246,24.155124,2.379249,1.410824,3.441129,2.364535,0.256967,0.056909,0.823504,0.129446,0.056175,8.931540e+08,7.951904e+10,0.598686,0.210802
min,23.460000,14.37000,0.990000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.960000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023000,1.233940e+09,1.366990e+11,0.000000,-0.299600
25%,25.965000,14.47500,1.485000,0.000000,0.014550,0.000000,0.000000,0.000000,-1.167500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.092075,1.254708e+09,1.419520e+11,0.000000,-0.144425
50%,27.435000,16.70500,1.830000,0.562500,0.022100,0.155000,4.950000,1.640000,-0.285000,2.395000,2.025000,0.171700,0.011500,0.520000,0.052650,0.126900,1.944755e+09,1.647440e+11,0.210000,-0.047000
75%,44.935000,24.93750,2.187500,1.449000,0.036050,0.496500,20.100000,3.637500,0.000000,5.290000,4.072500,0.385950,0.047100,1.197500,0.147525,0.139975,2.703780e+09,2.164412e+11,0.632500,0.051175
max,95.530000,43.05000,2.720000,2.421000,0.069800,1.056000,50.700000,4.710000,0.000000,6.790000,4.140000,0.513600,0.119400,1.670000,0.274200,0.143800,2.931480e+09,3.084100e+11,1.270000,0.208600


## 4. Observações

**Empresas baratas Brasileiras**

*Empresas que estejam valendo pouco comparadas ao tanto que ela gera de caixa.*



O EV/EBIT é um indicador financeiro que compara o Valor da Firma (EV ou Enterprise Value) com o Lucro Antes de Impostos e Taxas, o EBIT.

A principal função do EV/EBIT é ajudar a identificar quanto uma empresa custa em relação ao que ela produz a partir de sua atividade fim.

**EV/EBIT**

EV ---> Valor da Firma
Valor da Firma ---> Valor de mercado + Dív. Líquida

*Valor de mercado ---> Valor Ação (Cotação ou Preço atual) * Nro. de Ações emitidas*

EBIT ---> Lucro Líquido + Resultado Financeiro + Impostos

EV/EBIT ---> Valor de mercado (EV) / Lucro Antes de Juros e Impostos (EBIT)




É importante destacar que na fórmula EV/EBIT, o EV representa o valor de mercado da empresa somado à sua dívida líquida. Enquanto isso, o EBIT representa o nível de lucro operacional que a empresa produz em seu tamanho atual.

Nesse sentido, o EV/EBIT permite saber qual o potencial de geração de lucros de uma companhia. Vale lembrar que, para fins comparativos, ele deve ser utilizado apenas entre empresas do mesmo setor e, de preferência, que sejam concorrentes diretas.

**Exemplo:**

Uma empresa “A” e uma empresa “B” possuem, respectivamente, um EV/EBIT de 9,2 vezes e 8,9 vezes, seria possível afirmar que:

A empresa “A” está mais valorizada e melhor avaliada no mercado, podendo ser a melhor opção para a venda;

A empresa “B” está mais desvalorizada e pior avaliada no mercado, podendo ser a melhor opção para a compra.

Quando o EV/EBIT está elevado, existe uma indicação de que a empresa possui uma boa avaliação no mercado. Consequentemente, isso significa que suas ações estão valorizadas.

Enquanto isso, um EV/EBIT baixo demonstra que a empresa está sendo subavaliada. Com isso, possuem uma forte tendência de valorização ao longo do tempo, tornando a ação mais atrativa para a compra.

**Empresas boas Brasileiras**

O ROIC (Return Over Invested Capital), ou Retorno sobre o Capital Investido, é uma métrica utilizada para informar, em termos percentuais, quanto dinheiro uma empresa consegue gerar em razão do capital investido.

Com o ROIC, é possível ter uma visão geral sobre a performance financeira de um negócio.

**ROIC**

NOPAT ---> representa o lucro operacional após a distribuição de dividendos entre os acionistas

Valor Contábil ---> é a soma de todo capital investido na empresa.

ROIC ---> NOPAT / Valor Contábil

**Exemplo de uma empresa:**

Lucro Operacional: R$20 milhões

Valor Contábil do Capital Investido: R$80 milhões

Dividendos: R$5 milhões

NOPAT dessa empresa é igual a R$15 milhões

ROIC ---> 15.000.000 / 80.000.000 = 18,75%

(Lucro Operacional - Dividendos) / Valor Contábil do Capital Investido = ROIC



O parâmetro para avaliar se esse percentual é elevado ou não será o histórico da empresa e de suas concorrentes do mesmo setor.

Vale ressaltar que esse é um dado bem relevante pois demonstra o retorno sobre capital investido na empresa, mostrando muitas vezes qual companhia pode utilizar de forma mais eficiente o capital captado através de empréstimos, financiamentos ou acionistas.

Por conta dessas características, o ROIC é um dos indicadores financeiros mais utilizados por investidores na análise fundamentalista de ações.

Pois, muitos investidores acabam utilizando esse indicador na decisão sobre qual empresa realizar um investimento.